# EDA for AO - Water

## LIBS

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# import os

import plotly.express as px
pd.set_option('display.max_columns', None)

## FUNCTIONS

In [0]:
# Function to handle division and replace with 0 in case of errors
def safe_division(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        result = 0
    except FloatingPointError:
        result = 0
    return result


def plot_bar(dataframe,x,y):
    xdd=pd.DataFrame(dataframe.groupby(x).sum()[y]).reset_index()
    # print(xdd)
    fig=px.bar(data_frame=xdd.sort_values(by=y, ascending=False),x=x,y=y)
    return fig



In [0]:
# os.listdir()

## READING ALL DATA FROM SOURCE

In [0]:
cust=pd.read_csv("/dbfs/FileStore/shared_uploads/prem@loyalytics.in/assortment_optimization/confectionery/ao_cust_confectionery.csv")
attr=pd.read_csv("/dbfs/FileStore/shared_uploads/prem@loyalytics.in/assortment_optimization/confectionery/ao_attributes_confectionery.csv")
# attr['new_volume']=np.where(attr.units=='KG',attr.volume*1000,np.where(attr.units=='OZ',attr.volume*28.3495,np.where(attr.units=='LB',attr.volume*453.592,attr.volume*1)))

# attr['new_volume']=attr['new_volume'].astypeLB('object')

# attr['new_volume'] = attr['new_volume'].astype(str)+'G'

gp=pd.read_csv("/dbfs/FileStore/shared_uploads/prem@loyalytics.in/assortment_optimization/confectionery/ao_gp_confectionery.csv")
gp['gp_abs']=np.abs(gp['GP'])

df = pd.read_csv("/dbfs/FileStore/shared_uploads/prem@loyalytics.in/assortment_optimization/confectionery/ao_products_weekly_data_confectionery.csv")
df = pd.merge(df, gp[['material_id', 'new_buckets']], on='material_id', how='inner')

In [0]:
gp.head()

In [0]:
sale = df[['material_id', 'week_number', 'sale']]
vol = df[['material_id', 'week_number', 'vol']]
cwd = df[['material_id', 'week_number', 'cwd']]
store = df[['material_id', 'week_number', 'material_weekly_store_count']]

In [0]:
cust['vip_freq_pnt'] = (cust['vip_cust'] + cust['freq_cust']) / (cust['tot_vip'] + cust['tot_freq'])
cust

## FEATURE CREATION

In [0]:
df['SSW'] = df.apply(lambda row: safe_division(row['sale'], row['material_weekly_store_count']), axis=1)

In [0]:
fdf=df.copy()

## FINAL DATAFRAME

In [0]:
fdf.head()

#### UPDATING RECO NAMES

In [0]:
fdf['new_buckets'] = fdf['new_buckets'].str.replace('Maintain','SUPPORT_MORE_DIST') # high beta
fdf['new_buckets'] = fdf['new_buckets'].str.replace('Delist','DELIST')
fdf['new_buckets'] = fdf['new_buckets'].str.replace('Grow','MAINTAIN_OR_GROW_BY_PROMO') # low beta
fdf['new_buckets'] = fdf['new_buckets'].str.replace('Observe','OBSERVE')

gp['new_buckets'] = gp['new_buckets'].str.replace('Maintain','SUPPORT_MORE_DIST')
gp['new_buckets'] = gp['new_buckets'].str.replace('Delist','DELIST')
gp['new_buckets'] = gp['new_buckets'].str.replace('Grow','MAINTAIN_OR_GROW_BY_PROMO')
gp['new_buckets'] = gp['new_buckets'].str.replace('Observe','OBSERVE')

In [0]:
gp.new_buckets.value_counts()

### FINAL EDA DF FOR L12 AND L3M

In [0]:

############# FULL  dataframe ##############

agg_table_full = pd.DataFrame(
    fdf.groupby("material_id").agg(
        {
            "sale": "sum",
            "vol": "sum",
            "material_weekly_store_count": "mean",
            "cwd": "mean",'new_buckets':'max'
        }
    )
).reset_index()

agg_table_full.rename(columns={'material_weekly_store_count':'AVG_STORE_SELLING'},inplace=True)
agg_table_full.rename(columns={'cwd':'AVG_CWD'},inplace=True)
agg_table_full.columns = [x.lower() for x in agg_table_full.columns]



############# L3M dataframe ##############

agg_table_l3m = pd.DataFrame(
    fdf[fdf['week_number']>40].groupby("material_id").agg(
        {
            "sale": "sum",
            "vol": "sum",
            "material_weekly_store_count": "mean",
            "cwd": "mean",'new_buckets':'max'
        }
    )
).reset_index()

agg_table_l3m.rename(columns={'material_weekly_store_count':'AVG_STORE_SELLING'},inplace=True)
agg_table_l3m.rename(columns={'cwd':'AVG_CWD'},inplace=True)
agg_table_l3m.columns = [x.lower() for x in agg_table_l3m.columns]


In [0]:
agg_table_full

In [0]:
############ merging GP data ########################

# eda_agg_table_full=pd.merge(agg_table_full,attr,on='material_id',how='right')
eda_agg_table_full=pd.merge(agg_table_full,gp.drop(columns=['new_buckets'],axis=1),on='material_id',how='left')
eda_agg_table_full=pd.merge(eda_agg_table_full,cust.drop(columns=['material_name'],axis=1),on='material_id',how='left')
eda_agg_table_full['gp_flag']=np.where(eda_agg_table_full.GP<0,"Negative","Positive")

############ merging GP data L3m ########################

# eda_agg_table_l3m=pd.merge(agg_table_l3m,attr,on='material_id',how='right')
eda_agg_table_l3m=pd.merge(agg_table_full,gp.drop(columns=['new_buckets'],axis=1),on='material_id',how='left') ### GP data is still full here
eda_agg_table_l3m=pd.merge(eda_agg_table_l3m,cust.drop(columns=['material_name'],axis=1),on='material_id',how='left')

eda_agg_table_l3m['gp_flag']=np.where(eda_agg_table_l3m.GP<0,"Negative","Positive")

In [0]:
eda_agg_table_full.head(2)

## ALL PLOTS

### TLE and SSW

In [0]:
# pldf=fdf[fdf['material_id']==1545241]
# fig = px.scatter(pldf, x="SSW", y="cwd")
# fig.add_hline(y=pldf.cwd.mean())
# fig.add_vline(x=pldf.SSW.mean())

# fig.show()

In [0]:
fdf['sale']=fdf['sale'].astype('float')
fdf['vol']=fdf['vol'].astype('float')

In [0]:
gp_bub=fdf.groupby('material_id').sum()[['sale','vol']].reset_index()

temp=pd.merge(gp_bub,gp,on='material_id',how='inner')


# temp = pd.merge(temp,
#                 attr[['material_id', 'material_name','Brand']],
#                 left_on='material_id',
#                 right_on='material_id',
#                 how='inner')


temp['GP_ABS']=np.abs(temp['GP'])


temp['GP_FLAG']=np.where(temp.GP<0,"Negative","Positive")

In [0]:
temp.head()

In [0]:
# xd=pd.DataFrame(temp.groupby('Brand').sum()['sale']).reset_index()

### Sale, vol and other Bar Charts

In [0]:
eda_agg_table_full.head(1)

In [0]:
# plot_bar(eda_agg_table_full,'Brand','sale')

### BUBBLE CHARTS ALL

In [0]:
eda_agg_table_full.head(1)

#### SKU LEVEL BUBBLE


In [0]:
# pbldf=eda_agg_table_full.dropna(subset=['sale','vip_freq_pnt'])

# pbldf = pd.merge(pbldf, attr[['material_id', 'material_name']], on = 'material_id', how='inner')

# x_val='sale'
# y_val='vip_freq_pnt'
# color_val='gp_flag'
# hover_name_val='material_name'
# size_max_val=40
# size_val='gp_abs'

# fig = px.scatter(pbldf, x=x_val, y=y_val, size=np.abs(pbldf[size_val]), 
#                  color=color_val,
#                  hover_name=hover_name_val, log_x=False, log_y=False, size_max=size_max_val,title="ALL SKUs")

# fig.add_hline(y=pbldf[y_val].quantile(0.95))
# fig.add_vline(x=pbldf[x_val].quantile(0.95))

# fig.show()

In [0]:
temp = df.groupby('material_id')['sale'].sum().reset_index()

temp = pd.merge(temp, gp[['material_id', 'GP', 'new_buckets']], on='material_id', how='inner')

temp = pd.merge(temp,
                attr[['material_id', 'material_name','brand']],
                left_on='material_id',
                right_on='material_id',
                how='inner')

temp['GP_ABS']=np.abs(temp['GP'])

temp['GP_FLAG']=np.where(temp.GP<0, "Negative", "Positive")

temp = pd.merge(temp, cust[['material_id', 'tot_cust_perc', 'vip_freq_pnt']], on='material_id', how='left')
temp['tot_cust_perc'] = temp['tot_cust_perc'].fillna(0)
temp['vip_freq_pnt'] = temp['vip_freq_pnt'].fillna(0)
temp.rename(columns={'tot_cust_perc': 'cust_pnt'}, inplace=True)

In [0]:
fig = px.scatter(temp, x="sale", y="vip_freq_pnt", size="GP_ABS", 
                 color="GP_FLAG",
                 hover_name="material_name", log_x=False, size_max=40, color_discrete_map={"Positive": "#8993f8", "Negative": "#ec8272"})

fig.add_hline(y=temp.vip_freq_pnt.mean())
fig.add_vline(x=temp.sale.quantile(0.95))

fig.show()

In [0]:
temp["Quadrant"] = np.where(
    (temp["sale"] > temp["sale"].quantile(0.95))
    & (temp["vip_freq_pnt"] > temp["vip_freq_pnt"].mean()), 1,
    np.where(
        (temp["sale"] < temp["sale"].quantile(0.95))
        & (temp["vip_freq_pnt"] < temp["vip_freq_pnt"].mean()), 3,
    np.where(
        (temp["sale"] > temp["sale"].quantile(0.95))
        & (temp["vip_freq_pnt"] < temp["vip_freq_pnt"].mean()), 2,
    np.where(
        (temp["sale"] < temp["sale"].quantile(0.95))
        & (temp["vip_freq_pnt"] > temp["vip_freq_pnt"].mean()), 4,0
    )
)))

temp['pl_flag'] = np.where(temp['brand'] == "LULU PRIVATE LABEL", 1, 0)

In [0]:
display(temp.groupby(['Quadrant', 'material_id']).agg({'sale':'sum','cust_pnt':'sum','GP_ABS':'sum', 'vip_freq_pnt':'sum', 'pl_flag':'sum', 'GP':'sum'}).reset_index())

In [0]:
query = """
WITH cte AS (
    SELECT
        (CASE WHEN business_day BETWEEN "2022-02-01" AND "2023-01-31" THEN "LFL" ELSE "Current" END) AS year_info,
        material_id,
        ROUND(SUM(amount),0) AS sales
    FROM gold.pos_transactions AS t1
    JOIN gold.material_master AS t2 ON t1.product_id = t2.material_id
    WHERE business_day BETWEEN "2022-02-01" AND "2024-01-31"
    AND amount > 0
    AND quantity > 0
    AND category_name = "CONFECTIONERY"
    GROUP BY year_info, material_id
)

SELECT
    material_id,
    MAX(CASE WHEN year_info = "LFL" THEN sales ELSE 0 END) AS LFL_sales,
    MAX(CASE WHEN year_info = "Current" THEN sales ELSE 0 END) AS Current_sales,
    (Current_sales - LFL_sales)/LFL_sales AS growth
FROM cte
GROUP BY material_id
ORDER BY material_id
"""

e = spark.sql(query).toPandas()
e['growth'] = e['growth'].fillna(0)

In [0]:
temp = pd.merge(temp, e[['material_id', 'growth']], on='material_id', how='inner')

growth_positives = temp[temp['growth'] >= 0][['growth']].values
medium = np.percentile(growth_positives, 33)
high = np.percentile(growth_positives, 66)

temp["Growth_Flag"] = np.where(
    (temp["growth"] < 0), "Negative",
    np.where(
        (temp["growth"] > high), "High",
    np.where(
        (temp["growth"] > medium), "Medium", "New Launch"
    )
))

In [0]:
filtered_temp = temp[temp['brand'].str.contains('FERRERO')].reset_index()
fig = px.scatter(filtered_temp, x="sale", y="vip_freq_pnt", size="GP_ABS", 
                 color="Growth_Flag",
                 hover_name="material_name", log_x=False, size_max=40, color_discrete_map={"High": "#00CC96", "Medium": "#636EFA", "New Launch": "#AB63FA", "Negative": "#EF553B"})

fig.add_hline(y=temp.vip_freq_pnt.mean())
fig.add_vline(x=temp.sale.quantile(0.95))

fig.show()

In [0]:
filtered_temp["Quadrant"] = np.where(
    (filtered_temp["sale"] > filtered_temp["sale"].quantile(0.95))
    & (filtered_temp["vip_freq_pnt"] > filtered_temp["vip_freq_pnt"].mean()), 1,
    np.where(
        (filtered_temp["sale"] < filtered_temp["sale"].quantile(0.95))
        & (filtered_temp["vip_freq_pnt"] < filtered_temp["vip_freq_pnt"].mean()), 3,
    np.where(
        (filtered_temp["sale"] > filtered_temp["sale"].quantile(0.95))
        & (filtered_temp["vip_freq_pnt"] < filtered_temp["vip_freq_pnt"].mean()), 2,
    np.where(
        (filtered_temp["sale"] < filtered_temp["sale"].quantile(0.95))
        & (filtered_temp["vip_freq_pnt"] > filtered_temp["vip_freq_pnt"].mean()), 4,0
    )
)))

filtered_temp['pl_flag'] = np.where(filtered_temp['brand'] == "LULU PRIVATE LABEL", 1, 0)

In [0]:
display(filtered_temp.groupby(['Quadrant']).agg({'sale':'sum','cust_pnt':'sum','GP_ABS':'sum', 'vip_freq_pnt':'sum', 'pl_flag':'sum', 'GP':'sum', 'material_id':'nunique'}).reset_index())

#### SKU LEVEL BUBBLE - DELIST


In [0]:
# pbldf=eda_agg_table_full.dropna(subset=['sale','vip_freq_pnt'])
# # pbldf=pbldf[pbldf['new_buckets']=='Delist']
# x_val='sale'
# y_val='vip_freq_pntvip_freq_pnt'
# color_val='new_buckets'
# hover_name_val='material_name'
# size_max_val=40
# size_val='gp_abs'

# fig = px.scatter(pbldf, x=x_val, y=y_val, size=np.abs(pbldf[size_val]), 
#                  color=color_val,
#                  hover_name=hover_name_val, log_x=False, log_y=False, size_max=size_max_val,title="ALL SKUS BY BUCKETS")

# fig.add_hline(y=pbldf[y_val].quantile(0.95))
# fig.add_vline(x=pbldf[x_val].quantile(0.95))

# fig.show()

In [0]:
# pbldf.groupby('Brand').sum().reset_index()

#### AGG LEVEL BUBBLE - BY RECOMENDATIONS


In [0]:
# pbldf=eda_agg_table_full.dropna(subset=['sale','vip_freq_pnt'])

# # new_buckets
# # new_volume
# groupby_val='new_buckets'
# x_val='sale'
# y_val='vol'
# color_val='gp_flag'
# hover_name_val=groupby_val
# size_max_val=40
# size_val='gp_abs'

# pbldf=pbldf.groupby(groupby_val).sum().reset_index()
# pbldf['gp_flag']=np.where(pbldf.GP<0,"Negative","Positive")

# fig = px.scatter(pbldf, x=x_val, y=y_val, size=np.abs(pbldf[size_val]), 
#                  color=color_val,
#                  hover_name=hover_name_val, log_x=False, log_y=False, size_max=size_max_val,title="BY {}".format(groupby_val))

# fig.add_hline(y=pbldf[y_val].quantile(0.95))
# fig.add_vline(x=pbldf[x_val].quantile(0.95))

# fig.show()

In [0]:
# eda_agg_table_full.groupby('new_buckets').sum()['gp_abs']

In [0]:
def get_top_N_list(dataframe, start, end):
    listname = "list" + "_" + str(end)
#     print(listname)
    listname = pd.DataFrame(dataframe.groupby(
        'material_id').sum()['sale']).reset_index().sort_values(
            by=dataframe.columns[2], ascending=False)['material_id'][start:end].tolist()
    return listname

### TREND CHART DISTRIBUTIONS

#### TREND CHART MAINTAIN OR GROW

In [0]:
fdf.head(2)

In [0]:
# fdf.new_buckets.value_counts()
tx=fdf[fdf['new_buckets']=='MAINTAIN_OR_GROW_BY_PROMO']
tx=tx.groupby('material_id').sum()['sale'].reset_index().sort_values(by='sale',ascending=False)
tx_list=tx['material_id'].tolist()[0:20]

tbl=fdf[fdf['material_id'].isin(tx_list)]

tbl = pd.merge(tbl,
                attr[['material_id', 'material_name','brand']],
                left_on='material_id',
                right_on='material_id',
                how='inner')

fig = px.line(tbl, x="week_number", y="sale", color='material_name',width=900, height=600, title="SALES L52W")
fig.show()

fig2= px.line(tbl, x="week_number", y="cwd", color='material_name',width=900, height=600, title="CWD L52W")
fig2.show()

#### TREND CHART DELISTS

In [0]:
# # fdf.new_buckets.value_counts()
# tx=fdf[fdf['new_buckets']=='DELIST']
# tx=tx.groupby('material_id').sum()['sale'].reset_index().sort_values(by='sale',ascending=False)
# tx_list=tx['material_id'].tolist()[0:20]

# tbl=fdf[fdf['material_id'].isin(tx_list)]

# tbl = pd.merge(tbl,
#                 attr[['material_id', 'material_name','Brand']],
#                 left_on='material_id',
#                 right_on='material_id',
#                 how='inner')

# fig = px.line(tbl, x="week_number", y="sale", color='material_name',width=900, height=600)
# fig.show()

# fig2= px.line(tbl, x="week_number", y="cwd", color='material_name',width=900, height=600)
# fig2.show()

## END OF NOTEBOOK

In [0]:
fdf.head(1)

In [0]:
# # fdf.new_buckets.value_counts()
# tx=pd.merge(fdf,eda_agg_table_full[['brand','material_id']],on='material_id',how='left')
# # tx.head()
# tx=tx[tx['brand']=='AL AIN']
# tx=tx.groupby('material_id').sum()['sale'].reset_index().sort_values(by='sale',ascending=False)
# tx_list=tx['material_id'].tolist()[0:30]

# tbl=fdf[fdf['material_id'].isin(tx_list)]

# tbl = pd.merge(tbl,
#                 attr[['material_id', 'material_name','brand']],
#                 left_on='material_id',
#                 right_on='material_id',
#                 how='inner')

# fig = px.line(tbl, x="week_number", y="sale", color='material_name',width=900, height=600)
# fig.show()

# fig2= px.line(tbl, x="week_number", y="cwd", color='material_name',width=900, height=600)
# fig2.show()

In [0]:
display(eda_agg_table_full.groupby('new_buckets').sum()['sale'].reset_index())

In [0]:
# dl_br=pd.pivot_table(eda_agg_table_full,values='material_id',index='brand',columns='new_buckets',aggfunc='count')
# dl_br=dl_br.reset_index()
# dl_br['total_%_delist']=(dl_br['DELIST']/dl_br.sum(axis=1))*100
# dl_br

In [0]:
# tbl=store[store['material_id'].isin(get_top_N_list(sale,0,16))]

# tbl = pd.merge(tbl,
#                 attr[['material_id', 'material_name','Brand']],
#                 left_on='material_id',
#                 right_on='material_id',
#                 how='inner')

# fig = px.line(tbl, x="week_number", y="store", color='material_name',width=900, height=600)
# fig.show()

### interactions

In [0]:
tbl=sale[sale['material_id'].isin(get_top_N_list(sale,0,16))]

tbl = pd.merge(tbl,
                attr[['material_id', 'material_name']],
                left_on='material_id',
                right_on='material_id',
                how='inner')

fig = px.line(tbl, x="week_number", y="sale", color='material_name',width=900, height=600)
fig.show()

### Brand level

In [0]:
query = """
WITH total_cust AS (SELECT COUNT(DISTINCT t3.customer_id) AS tot_cust,
                            COUNT(DISTINCT CASE WHEN segment = 'VIP' THEN t3.customer_id END) AS tot_vip,
                            COUNT(DISTINCT CASE WHEN segment = 'Frequentist' THEN t3.customer_id END) AS tot_freq
                    FROM gold.pos_transactions AS t1
                    JOIN gold.material_master AS t2
                    ON t1.product_id = t2.material_id
                    JOIN analytics.customer_segments AS t3
                    ON t1.customer_id = t3.customer_id
                    WHERE business_day BETWEEN '2023-02-01' AND '2024-01-30'
                    AND category_name = 'CONFECTIONERY'
                    AND ROUND(amount,0) > 0
                    AND quantity > 0
                    AND month_year = '202401'
                    AND t3.country = 'uae'
                    AND key = 'rfm'
                    AND channel = 'pos'
)

SELECT brand,
        COUNT(DISTINCT t3.customer_id) AS cust,
        COUNT(DISTINCT CASE WHEN segment = 'VIP' THEN t3.customer_id END) AS vip_cust,
        COUNT(DISTINCT CASE WHEN segment = 'Frequentist' THEN t3.customer_id END) AS freq_cust,
        tot_cust, tot_vip, tot_freq,
        (cust/tot_cust) AS tot_cust_perc,
        (vip_cust/tot_vip) AS vip_cust_perc,
        (freq_cust/tot_freq) AS freq_cust_perc
FROM total_cust, gold.pos_transactions AS t1
JOIN gold.material_master AS t2
ON t1.product_id = t2.material_id
JOIN analytics.customer_segments AS t3
ON t1.customer_id = t3.customer_id
WHERE business_day BETWEEN '2023-02-01' AND '2024-01-30'
AND category_name = 'CONFECTIONERY'
AND ROUND(amount,0) > 0
AND quantity > 0
AND month_year = '202401'
AND t3.country = 'uae'
AND key = 'rfm'
AND channel = 'pos'
GROUP BY brand, tot_cust, tot_vip, tot_freq
ORDER BY brand
"""

p = spark.sql(query).toPandas()
p['vip_freq_pnt'] = (p['vip_cust'] + p['freq_cust']) / (p['tot_vip'] + p['tot_freq'])

In [0]:
gp_bub2 = pd.merge(fdf,
                attr[['material_id', 'material_name','brand']],
                left_on='material_id',
                right_on='material_id',
                how='inner')

gp_bub=gp_bub2.groupby('brand').sum()[['sale','vol']].reset_index()

gp2 = pd.merge(gp,
                attr[['material_id', 'material_name','brand']],
                left_on='material_id',
                right_on='material_id',
                how='inner')

gp3=gp2.groupby('brand').sum()['GP']

temp = pd.merge(gp_bub, gp3, on='brand',how='inner')
temp = pd.merge(temp, p[['brand', 'vip_freq_pnt']], on='brand',how='inner')

temp['GP_ABS']=np.abs(temp['GP'])

temp['GP_FLAG']=np.where(temp.GP<0,"Negative","Positive")

In [0]:
fig = px.scatter(temp, x="sale", y="vip_freq_pnt", size="GP_ABS", 
                 color="GP_FLAG",
                 hover_name="brand", log_x=False, size_max=40)

fig.add_hline(y=temp.vip_freq_pnt.mean())
fig.add_vline(x=temp.sale.quantile(0.95))

fig.show()

In [0]:
query = """
WITH cte AS (
    SELECT
        (CASE WHEN business_day BETWEEN "2022-02-01" AND "2023-01-31" THEN "LFL" ELSE "Current" END) AS year_info,
        brand,
        ROUND(SUM(amount),0) AS sales
    FROM gold.pos_transactions AS t1
    JOIN gold.material_master AS t2 ON t1.product_id = t2.material_id
    WHERE business_day BETWEEN "2022-02-01" AND "2024-01-31"
    AND amount > 0
    AND quantity > 0
    AND category_name = "CONFECTIONERY"
    GROUP BY year_info, brand
)

SELECT
    brand,
    MAX(CASE WHEN year_info = "LFL" THEN sales ELSE 0 END) AS LFL_sales,
    MAX(CASE WHEN year_info = "Current" THEN sales ELSE 0 END) AS Current_sales,
    (Current_sales - LFL_sales)/LFL_sales AS growth
FROM cte
GROUP BY brand
ORDER BY brand
"""

r = spark.sql(query).toPandas()
r['growth'] = r['growth'].fillna(0)

In [0]:
temp = pd.merge(temp, r[['brand', 'growth']], on='brand', how='inner')

growth_positives = temp[temp['growth'] >= 0][['growth']].values
medium = np.percentile(growth_positives, 33)
high = np.percentile(growth_positives, 66)

temp["Growth_Flag"] = np.where(
    (temp["growth"] < 0), "Negative",
    np.where(
        (temp["growth"] > high), "High",
    np.where(
        (temp["growth"] > medium), "Medium", "Low"
    )
))

In [0]:
filtered_temp = temp[temp['brand'].str.contains('FERRERO')]
fig = px.scatter(filtered_temp, x="sale", y="vip_freq_pnt", size="GP_ABS", 
                 color="Growth_Flag",
                 hover_name="brand", log_x=False, size_max=40, color_discrete_map={"High": "#00CC96", "Medium": "#636EFA", "Low": "#AB63FA", "Negative": "#EF553B"})

fig.add_hline(y=temp.vip_freq_pnt.mean())
fig.add_vline(x=temp.sale.quantile(0.95))

fig.show()

In [0]:
temp["Manufacturer"] = np.where(
    (temp["brand"].str.contains("MARS")), "MARS",
    np.where(
        (temp["brand"].str.contains("FERRERO")), "FERRERO",
    np.where(
        (temp["brand"].str.contains("NESTLE")), "NESTLE", "OTHERS"
    )
))

In [0]:
fig = px.scatter(temp, x="sale", y="vip_freq_pnt", size="GP_ABS", 
                 color="Manufacturer",
                 hover_name="brand", log_x=False, size_max=40)

fig.add_hline(y=temp.vip_freq_pnt.mean())
fig.add_vline(x=temp.sale.quantile(0.95))

fig.show()

In [0]:
temp["Quadrant"] = np.where(
    (temp["sale"] > temp["sale"].quantile(0.95))
    & (temp["vip_freq_pnt"] > temp["vip_freq_pnt"].mean()), 1,
    np.where(
        (temp["sale"] < temp["sale"].quantile(0.95))
        & (temp["vip_freq_pnt"] < temp["vip_freq_pnt"].mean()), 3,
    np.where(
        (temp["sale"] > temp["sale"].quantile(0.95))
        & (temp["vip_freq_pnt"] < temp["vip_freq_pnt"].mean()), 2,
    np.where(
        (temp["sale"] < temp["sale"].quantile(0.95))
        & (temp["vip_freq_pnt"] > temp["vip_freq_pnt"].mean()), 4,0
    )
)))

In [0]:
display(temp.groupby(['Quadrant', 'brand', 'Manufacturer', 'Growth_Flag']).agg({'sale':'sum', 'GP_ABS':'sum', 'growth':'sum', 'vip_freq_pnt':'sum'}).reset_index())

#### Price calculation

In [0]:
df.head()

In [0]:
price_df=df[df['week_number']>40].groupby('material_id').sum()[['sale','vol']].reset_index()
price_df['price']=price_df.sale/price_df.vol

In [0]:
px.histogram(price_df,y='price')

In [0]:
xd=pd.merge(price_df,attr, on='material_id',how='right')

In [0]:
display(xd)

In [0]:
attr.head()

In [0]:
xd=sale.groupby('material_id').sum()['sale'].reset_index()
xd=pd.merge(xd,attr, on='material_id',how='right')

In [0]:
display(xd)

In [0]:
# t=pd.DataFrame(xd.groupby('new_volume').sum()['sale']).reset_index()
# t['new_volume']=t['new_volume'].astype('object')

# t['new_volume'] = t['new_volume'].astype(str)+'ML'
# t
# # print(t.dtypes)
# plot_bar(t,'new_volume','sale')


#### quadrant values and GP

In [0]:
eda_agg_table_full.head(1)

In [0]:
eda_agg_table_full["Quadrant"] = np.where(
    (eda_agg_table_full["sale"] > eda_agg_table_full["sale"].quantile(0.95))
    & (eda_agg_table_full["tot_cust_perc"] > eda_agg_table_full["tot_cust_perc"].mean()), 1,
    np.where(
        (eda_agg_table_full["sale"] < eda_agg_table_full["sale"].quantile(0.95))
        & (eda_agg_table_full["tot_cust_perc"] < eda_agg_table_full["tot_cust_perc"].mean()), 3,
    np.where(
        (eda_agg_table_full["sale"] > eda_agg_table_full["sale"].quantile(0.95))
        & (eda_agg_table_full["tot_cust_perc"] < eda_agg_table_full["tot_cust_perc"].mean()), 2,
    np.where(
        (eda_agg_table_full["sale"] < eda_agg_table_full["sale"].quantile(0.95))
        & (eda_agg_table_full["tot_cust_perc"] > eda_agg_table_full["tot_cust_perc"].mean()), 4,0
    )
)))

eda_agg_table_full = pd.merge(eda_agg_table_full, attr[['material_id', 'brand']], on='material_id', how='inner')

eda_agg_table_full['pl_flag'] = np.where(eda_agg_table_full['brand'] == "LULU PRIVATE LABEL", 1, 0)

In [0]:
display(eda_agg_table_full.groupby('Quadrant').agg({'sale':'sum','tot_cust_perc':'mean','gp_abs':'sum', 'material_id':'count', 'pl_flag':'sum'}).reset_index())

In [0]:
fdf.merge(eda_agg_table_full)

In [0]:
display(eda_agg_table_full.groupby('new_buckets').sum()['sale'].reset_index())


### Private label view

In [0]:
pbldf=eda_agg_table_full.dropna(subset=['sale','vip_freq_pnt'])
pbldf = pd.merge(pbldf, attr[['material_id', 'material_name', 'brand']], on='material_id', how='inner')
pbldf['pl_flag'] = np.where(pbldf['brand'] == "LULU PRIVATE LABEL", "PL", "Other")
x_val='sale'
y_val='vip_freq_pnt'
color_val='pl_flag'
hover_name_val='material_name'
size_max_val=40
size_val='gp_abs'

fig = px.scatter(pbldf, x=x_val, y=y_val, size=np.abs(pbldf[size_val]), 
                 color=color_val,
                 hover_name=hover_name_val, log_x=False, log_y=False, size_max=size_max_val,title="LULU PL SKUs AND OTHERS")

fig.add_hline(y=pbldf[y_val].quantile(0.95))
fig.add_vline(x=pbldf[x_val].quantile(0.95))

fig.show()

In [0]:
pbldf=eda_agg_table_full.dropna(subset=['sale','vip_freq_pnt'])
pbldf = pd.merge(pbldf, attr[['material_id', 'material_name', 'brand']], on='material_id', how='inner')
pbldf=pbldf[pbldf['brand']=='LULU PRIVATE LABEL']
x_val='sale'
y_val='vip_freq_pnt'
color_val='new_buckets'
hover_name_val='material_name'
size_max_val=40
size_val='gp_abs'

fig = px.scatter(pbldf, x=x_val, y=y_val, size=np.abs(pbldf[size_val]), 
                 color=color_val,
                 hover_name=hover_name_val, log_x=False, log_y=False, size_max=size_max_val,title="LULU PL SKUS BY BUCKETS")

fig.add_hline(y=pbldf[y_val].quantile(0.95))
fig.add_vline(x=pbldf[x_val].quantile(0.95))

fig.show()

### moving avg

In [0]:

df_ma = pd.DataFrame(columns=fdf.columns)

for i in fdf.material_id.unique():
    # print(tx_list)  # Instead of tx.columns
    temp = fdf[fdf['material_id'] == i].sort_values(by='week_number')
    temp['MA4W'] = temp['sale'].rolling(4).mean()
    df_ma=pd.concat([temp,df_ma])
    # print(i)

In [0]:
df_ma['sale']=df_ma['sale'].astype('float64')

In [0]:
# fdf.new_buckets.value_counts()
tx=df_ma[df_ma['new_buckets']=='OBSERVE']

tx=tx.groupby('material_id').sum()['sale'].reset_index().sort_values(by='sale',ascending=False)
tx_list=tx['material_id'].tolist()[0:20]
print(tx_list)

tbl=df_ma[df_ma['material_id'].isin(tx_list)]


tbl = pd.merge(tbl,
                attr[['material_id', 'material_name','Brand']],
                left_on='material_id',
                right_on='material_id',
                how='inner')

fig = px.line(tbl, x="week_number", y="MA4W", color='material_name',width=900, height=600)
fig.show()


# fig2= px.line(tbl, x="week_number", y="cwd", color='material_name',width=900, height=600)
# fig2.show()



### Delist by Attributes

In [0]:
# eda_agg_table_full.Type.value_counts()

In [0]:
pbldf=eda_agg_table_full.dropna(subset=['sale','vip_freq_pnt'])
pbldf=pbldf[pbldf['new_buckets']=='DELIST']
x_val='sale'
y_val='vip_freq_pnt'
color_val='Type'
hover_name_val='material_name'
size_max_val=40
size_val='gp_abs'

fig = px.scatter(pbldf, x=x_val, y=y_val, size=np.abs(pbldf[size_val]), 
                 color=color_val,
                 hover_name=hover_name_val, log_x=False, log_y=False, size_max=size_max_val,title="DELIST BY ATTRIBUTES-  SKUs")



fig.add_hline(y=pbldf[y_val].quantile(0.95))
fig.add_vline(x=pbldf[x_val].quantile(0.95))


fig.show()


In [0]:

############# L3M Previous dataframe ##############

agg_table_l3m_before = pd.DataFrame(
    fdf[(fdf['week_number']>28)&(fdf['week_number']<41)].groupby("material_id").agg(
        {
            "sale": "sum",
            "vol": "sum",
            "material_weekly_store_count": "mean",
            "cwd": "mean",'new_buckets':'max'
        }
    )
).reset_index()

agg_table_l3m_before.rename(columns={'material_weekly_store_count':'AVG_STORE_SELLING'},inplace=True)
agg_table_l3m_before.rename(columns={'cwd':'AVG_CWD'},inplace=True)
agg_table_l3m_before.columns = [x.lower() for x in agg_table_l3m_before.columns]

In [0]:
display(eda_agg_table_full)

## SLIDES CHARTS

### SLIDE - 6

In [0]:

tt = eda_agg_table_full.groupby('material_id').sum()['sale'].reset_index().sort_values(by='sale', ascending=False)
tt['cum_sum'] = np.round(100 * (tt.sale.cumsum() / tt['sale'].sum()), 2)
tt

In [0]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np

# Assuming eda_agg_table_full is your DataFrame

tt = eda_agg_table_full.groupby('material_id').sum()['gp_abs'].reset_index().sort_values(by='gp_abs', ascending=False)
tt['cum_sum'] = np.round(100 * (tt.gp_abs.cumsum() / tt['gp_abs'].sum()), 2)
tt['material_id']=tt['material_id'].astype('str')
fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(y=tt['gp_abs'], x=tt['material_id'], name="Brand GP"), secondary_y=False)

fig.add_trace(go.Scatter(x=tt['material_id'], y=tt['cum_sum'],
                         mode='lines', name="GP Cumulative Contri"), secondary_y=True)

# Add figure title
fig.update_layout(
    title_text="gp_abs dist"
)

# Set x-axis title
fig.update_xaxes(title_text="xaxis title")

# Set y-axes titles
fig.update_yaxes(title_text="material_id gp_abs", secondary_y=False)
fig.update_yaxes(title_text="% Cumulative Contribution", secondary_y=True)
fig.update_layout(showlegend=False)
fig.update_layout(autosize=False, width=900, height=400)
# fig.add_hline(secondary_y=80)
fig.show()

In [0]:
gp.display()

###Data Tables

In [0]:
%sql
-- Manufacturer-Material Group Level - Monthly RPC
WITH cte AS (
  SELECT
      CASE WHEN brand LIKE "%MARS%" THEN "MARS"
      WHEN brand LIKE "%FERRERO%" THEN "FERRERO"
      WHEN brand LIKE "%NESTLE%" THEN "NESTLE"
      ELSE "OTHERS" END AS manufacturer,
      segment,
      material_group_name,
      ROUND(SUM(amount),0)/12 AS spend_per_month,
      COUNT(DISTINCT mobile) AS customers,
      ROUND(spend_per_month/customers,2) AS RPC_monthly
  FROM gold.pos_transactions AS t1
  JOIN gold.material_master AS t2 ON t1.product_id = t2.material_id
  JOIN analytics.customer_segments AS t3 ON t1.customer_id = t3.customer_id
  WHERE business_day BETWEEN "2023-02-01" AND "2024-01-30"
  AND category_name = "CONFECTIONERY"
  AND amount > 0
  AND quantity > 0
  AND key = 'rfm'
  AND channel = 'pos'
  AND t3.country = 'uae'
  AND month_year = '202401'
  GROUP BY manufacturer, segment, material_group_name
)

SELECT
    manufacturer,
    material_group_name,
    MAX(CASE WHEN segment = "Frequentist" THEN RPC_monthly END) AS Frequentist,
    MAX(CASE WHEN segment = "VIP" THEN RPC_monthly END) AS VIP,
    MAX(CASE WHEN segment = "Moderate" THEN RPC_monthly END) AS Moderate,
    MAX(CASE WHEN segment = "Splurger" THEN RPC_monthly END) AS Splurger,
    MAX(CASE WHEN segment = "Lapser" THEN RPC_monthly END) AS Lapser,
    MAX(CASE WHEN segment = "Slipping Loyalist" THEN RPC_monthly END) AS Slipping_Loyalist
FROM cte
GROUP BY manufacturer, material_group_name

In [0]:
%sql
-- Manufacturer-Overall Level - Monthly RPC
WITH cte AS (
  SELECT
      CASE WHEN brand LIKE "%MARS%" THEN "MARS"
      WHEN brand LIKE "%FERRERO%" THEN "FERRERO"
      WHEN brand LIKE "%NESTLE%" THEN "NESTLE"
      ELSE "OTHERS" END AS manufacturer,
      segment,
      ROUND(SUM(amount),0)/12 AS spend_per_month,
      COUNT(DISTINCT mobile) AS customers,
      ROUND(spend_per_month/customers,2) AS RPC_monthly
  FROM gold.pos_transactions AS t1
  JOIN gold.material_master AS t2 ON t1.product_id = t2.material_id
  JOIN analytics.customer_segments AS t3 ON t1.customer_id = t3.customer_id
  WHERE business_day BETWEEN "2023-02-01" AND "2024-01-30"
  AND category_name = "CONFECTIONERY"
  AND amount > 0
  AND quantity > 0
  AND key = 'rfm'
  AND channel = 'pos'
  AND t3.country = 'uae'
  AND month_year = '202401'
  GROUP BY manufacturer, segment
)

SELECT
    manufacturer,
    "OVERALL" AS material_group_name,
    MAX(CASE WHEN segment = "Frequentist" THEN RPC_monthly END) AS Frequentist,
    MAX(CASE WHEN segment = "VIP" THEN RPC_monthly END) AS VIP,
    MAX(CASE WHEN segment = "Moderate" THEN RPC_monthly END) AS Moderate,
    MAX(CASE WHEN segment = "Splurger" THEN RPC_monthly END) AS Splurger,
    MAX(CASE WHEN segment = "Lapser" THEN RPC_monthly END) AS Lapser,
    MAX(CASE WHEN segment = "Slipping Loyalist" THEN RPC_monthly END) AS Slipping_Loyalist
FROM cte
GROUP BY manufacturer

In [0]:
%sql
-- All Brands-Material Group Level - Sales Contribution
WITH cte AS (
  SELECT
      segment,
      material_group_name,
      ROUND(SUM(amount),0) AS sales
  FROM gold.pos_transactions AS t1
  JOIN gold.material_master AS t2 ON t1.product_id = t2.material_id
  JOIN analytics.customer_segments AS t3 ON t1.customer_id = t3.customer_id
  WHERE business_day BETWEEN "2023-02-01" AND "2024-01-30"
  AND category_name = "CONFECTIONERY"
  AND amount > 0
  AND quantity > 0
  AND key = 'rfm'
  AND channel = 'pos'
  AND t3.country = 'uae'
  AND month_year = '202401'
  GROUP BY segment, material_group_name
)

SELECT
    material_group_name,
    MAX(CASE WHEN segment = "Frequentist" THEN sales END) AS Frequentist,
    MAX(CASE WHEN segment = "VIP" THEN sales END) AS VIP,
    MAX(CASE WHEN segment = "Moderate" THEN sales END) AS Moderate,
    MAX(CASE WHEN segment = "Splurger" THEN sales END) AS Splurger,
    MAX(CASE WHEN segment = "Lapser" THEN sales END) AS Lapser,
    MAX(CASE WHEN segment = "Slipping Loyalist" THEN sales END) AS Slipping_Loyalist
FROM cte
GROUP BY material_group_name

In [0]:
%sql
-- Segment Level Overview Metrics
SELECT
    segment,
    COUNT(DISTINCT mobile) AS total_customers,
    COUNT(DISTINCT CASE WHEN category_name = "CONFECTIONERY" THEN mobile END) AS total_confectionery_customers,
    COUNT(DISTINCT CASE WHEN category_name = "CONFECTIONERY" AND brand LIKE "%FERRERO%" THEN mobile END) AS total_confectionery_ferrero_customers,
    ROUND(SUM(amount),0) AS sales,
    ROUND(SUM(DISTINCT CASE WHEN category_name = "CONFECTIONERY" THEN amount END),0) AS total_confectionery_sales,
    ROUND(SUM(DISTINCT CASE WHEN category_name = "CONFECTIONERY" AND brand LIKE "%FERRERO%" THEN amount END),0) AS total_confectionery_ferrero_sales
FROM gold.pos_transactions AS t1
JOIN gold.material_master AS t2 ON t1.product_id = t2.material_id
JOIN analytics.customer_segments AS t3 ON t1.customer_id = t3.customer_id
WHERE business_day BETWEEN "2023-02-01" AND "2024-01-30"
AND amount > 0
AND quantity > 0
AND key = 'rfm'
AND channel = 'pos'
AND t3.country = 'uae'
AND month_year = '202401'
GROUP BY segment

In [0]:
%sql
-- Manufacturer-Material Group Level - Total Sales
WITH cte AS (
  SELECT
      CASE WHEN brand LIKE "%MARS%" THEN "MARS"
      WHEN brand LIKE "%FERRERO%" THEN "FERRERO"
      WHEN brand LIKE "%NESTLE%" THEN "NESTLE"
      ELSE "OTHERS" END AS manufacturer,
      segment,
      material_group_name,
      ROUND(SUM(amount),0) AS sales
  FROM gold.pos_transactions AS t1
  JOIN gold.material_master AS t2 ON t1.product_id = t2.material_id
  JOIN analytics.customer_segments AS t3 ON t1.customer_id = t3.customer_id
  WHERE business_day BETWEEN "2023-02-01" AND "2024-01-30"
  AND category_name = "CONFECTIONERY"
  AND amount > 0
  AND quantity > 0
  AND key = 'rfm'
  AND channel = 'pos'
  AND t3.country = 'uae'
  AND month_year = '202401'
  GROUP BY manufacturer, segment, material_group_name
)

SELECT
    manufacturer,
    material_group_name,
    MAX(CASE WHEN segment = "VIP" THEN sales END) AS VIP,
    MAX(CASE WHEN segment = "Frequentist" THEN sales END) AS Frequentist,
    MAX(CASE WHEN segment = "Splurger" THEN sales END) AS Splurger,
    MAX(CASE WHEN segment = "Slipping Loyalist" THEN sales END) AS Slipping_Loyalist,
    MAX(CASE WHEN segment = "Moderate" THEN sales END) AS Moderate,
    MAX(CASE WHEN segment = "Lapser" THEN sales END) AS Lapser
FROM cte
GROUP BY manufacturer, material_group_name

In [0]:
%sql
SELECT brand, ROUND(SUM(amount),0) AS sales
FROM gold.pos_transactions AS t1
JOIN gold.material_master AS t2 ON t1.product_id = t2.material_id
WHERE business_day BETWEEN "2023-02-01" AND "2024-01-30"
-- AND category_name = "CONFECTIONERY"
AND amount > 0
AND quantity > 0
AND brand LIKE "%FERRERO%"
GROUP BY brand